In [ ]:
# import whisper

# model = whisper.load_model("turbo")

# # load audio and pad/trim it to fit 30 seconds
# audio = whisper.load_audio("grabacion_temp.wav")
# audio = whisper.pad_or_trim(audio)

# # make log-Mel spectrogram and move to the same device as the model
# mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)

# # detect the spoken language
# _, probs = model.detect_language(mel)
# print(f"Detected language: {max(probs, key=probs.get)}")


# # decode the audio forcing English language
# options = whisper.DecodingOptions(language="en")
# result = whisper.decode(model, mel, options)


# # print the recognized text
# print(result.text)

In [ ]:
#! pip install pyaudio

In [3]:
import pyaudio
import wave
import audioop
import whisper

# Recording parameters
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000  # For speech, 16kHz is usually sufficient
CHUNK = 1024
SILENCE_THRESHOLD = 1000  # Adjust based on your mic sensitivity (lower = more sensitive)
SILENCE_DURATION = 3      # Seconds of silence required to stop recording

def record_until_silence(filename):
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Recording...")
    frames = []
    silent_chunks = 0
    required_silent_chunks = int(RATE / CHUNK * SILENCE_DURATION)

    while True:
        data = stream.read(CHUNK)
        frames.append(data)
        # Compute RMS (root mean square) to estimate volume
        rms = audioop.rms(data, 2)
        if rms < SILENCE_THRESHOLD:
            silent_chunks += 1
        else:
            silent_chunks = 0

        if silent_chunks > required_silent_chunks:
            break

    print("Finished recording.")
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save the recorded data as a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

# Record audio from the mic and save it to a file
record_until_silence("recording.wav")

# Load and transcribe with Whisper
model = whisper.load_model("turbo")

# Load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("recording.wav")
audio = whisper.pad_or_trim(audio)

# Create a log-Mel spectrogram and move it to the same device as the model
mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)

# Detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# Decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# Print the recognized text
print(result.text)


Recording...
Finished recording.
Detected language: en
Hello, my name is Bruno. I'm from Braga. I like to eat meat.
